<a href="https://colab.research.google.com/github/MaxGarnica/Trabajos_SimulacionII/blob/main/Problemas_de_l%C3%ADneas_de_espera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Maximiliano Garnica López*

*Tarea*

*Simulación II*

# Problemas de líneas de espera

**Instrucciones**

Elabore esta tarea en una notebook independiente, registrando sus resultados o las salidas de los programas. En todos los casos en que sea posible, resuelva los problemas mediante simulación y después verifique los resultados con las fórmulas correspondientes.

In [9]:
import random
from matplotlib.pyplot import *
import numpy as np
import pandas as pd
from statistics import *
from math import inf as Infinity
from time import time
import math

## 1. En la biblioteca

Un encargado de la biblioteca es el único en el mostrador durante todo su turno de trabajo. Las llegadas al mostrador siguen una distribución de Poisson con una media de 8 por hora. Cada usuario de la biblioteca es atendido de uno en uno, y el tiempo de servicio sigue una distribución exponencial con una media de 5 minutos.

### a) ¿Cuál es la probabilidad de que se forme cola?

En este inciso realizaremos una función que nos regrese la probabilidad dependiendo del número de llegadas.

In [19]:
def pmm1_ll(lamda,mu,Num_Pkts):
  count = 0                           # Contador de paquetes simulados
  t = 0
  N = 0                               # Variable de estado: numero de paquetes en el sistema

  tll = random.expovariate(lamda)     # Tiempo de llegada
  ts = Infinity                       # Tiempo de salida
  t_evento_anterior = 0.0             # tiempo del último evento

  # Variables de salida
  tll_Data = []                       # Lista de tiempos de llegada
  ts_Data = []                        # Lista de tiempos de salida
  r_Data = []                         # Lista de retrasos individuales

  Data = {}                           # Diccionario

  while count < Num_Pkts:
  #while t < T:
      if tll < ts:                                 # Evento de llegada
          t = tll
          
          # Longitud del intervalo entre llegadas
          delta = t - t_evento_anterior
          if N in Data: 
              Data[N] += delta
          else:
              Data[N] = delta
          t_evento_anterior = t
          
          tll_Data.append(t)
          N = N + 1.0
          tll = t + random.expovariate(lamda) 
          if N == 1:
              ts = t + random.expovariate(mu)
      else:                                         # Evento de salida
          t = ts
          
          # Longitud del intervalo de tiempo
          delta = t - t_evento_anterior
          if N in Data:
              Data[N] += delta
          else:
              Data[N] = delta
          t_evento_anterior = t
          
          ts_Data.append(t)
          N = N - 1.0
          count = count + 1                         # Paquetes simulados
          if N > 0:
              ts = t + random.expovariate(mu)
          else:
              ts = Infinity

  ##################################### se procesan los datos de salida ########################################

  # Estimación de la probabilidad de P(N=k):
  for (key, value) in Data.items():
      Data[key] = value/t

  # Se verifica que la probabilidad total es 1.0

  pt=sum(Data.values())
  #print('Suma de probabilidades =', sum(Data.values()))

  # Se verifican los valores de expectación
  media = 0.0
  md=[]
  
  #print('\n')
  #print('Lista de Probabilidades\n')
  #print('No. Llegadas','\t','Probabilidad')
  for (key, value) in Data.items():
      media = media + key*value
      md.append(media)
      #print(key,'\t\t', round(value, 4))

  #print('\n')
  #print('E[N] = ', media)

  return(md)

Como las llegadas tienen una distribucion poisson, los servicios una distribucion exponecial y solo tenemos un servidor, tenemos un modelo de la forma: **M/M/1**

El promedio de llegadas es de $λ=\frac{8\text{ llegadas}}{1\text{ hora}}$, pero haremos la conversion a llegadas por minuto.

$λ=\frac{8\text{ llegadas}}{1\text{ hora}}(\frac{1\text{ hora}}{60 \text{ minuto}})=\frac{2}{15} \frac{\text{llegadas}}{\text{minuto}}$

El promedio de servicios es de $μ=\frac{1 \text{ servicio}}{5 \text{ minuto}}$

Como no tenemos especificaciones sobre cuanto dura el turno del empleado simularemos $100$ usuarios.

Al solo tener un servidor sabemos que si llegan más de un usuario a la vez tendremos cola, por lo que la probabilidad la obtendremos de la siguiente manera.

$P(x>1)=1-P(x<=1)$

In [104]:
Prob = pmm1_ll(2/15,1/5,100)       #Arreglo que guarda las probabilidades del numero de llegadas

Pc = 1 - ( Prob[1] + Prob[2] )    #Probabilidad de que se forme cola = Probabilidad total - ( Probabilidad de cero llegadas + Probabilidad de tener una llegada )

print("Probabilidad de que se forme cola es de %.4f" %Pc)

Probabilidad de que se forme cola es de 0.2532


### Funcion que devuelve W, Wq, L y Lq

Ahora realizaremos una funcion en la que regresaremos las variables:

* $W$ : Tiempo medio de estancia de los clientes en el sistema.
* $W_q$ : Tiempo medio de espera de los clientes en la cola.
* $L$ : Número medio de clientes en el sistema.
* $L_q$ : Número medio de clientes en la cola.


In [26]:
def mm1_var(lamda,mu,Num_Pkts):
  count = 0                           # Contador de paquetes simulados
  t = 0
  N = 0                               # Variable de estado: numero de paquetes en el sistema

  tll = random.expovariate(lamda)     # Tiempo de llegada
  ts = Infinity                       # Tiempo de salida

  t_evento_anterior = 0.0             # tiempo del último evento

  # Variables de salida
  tll_Data = []                       # Lista de tiempos de llegada
  ts_Data = []                        # Lista de tiempos de salida
  ti_Data = []                        # Lista de tiempos de inicio de servicio
  tc_Data = []                        # Lista de tiempos en la cola
  r_Data = []                         # Lista de retrasos individuales

  ti_Data.append(tll)

  while count < Num_Pkts:
      if tll < ts:                                  # Evento de llegada
          t = tll
          tll_Data.append(t)
          
          t_evento_anterior = t
          
          N = N + 1.0
          tll = t + random.expovariate(lamda) 

          if N == 1:
              ts = t + random.expovariate(mu)

      else:                                         # Evento de salida
          t = ts
          ts_Data.append(t)

          t_evento_anterior = t

          N = N - 1.0
          count = count + 1                         # Paquetes simulados
          if N > 0:
              ts = t + random.expovariate(mu)
          else:
              ts = Infinity

  for j in range(Num_Pkts):                         # Tiempo de inicio del servicio
    if tll_Data[j+1]<ts_Data[j]:
      ti_Data.append(ts_Data[j])
    else:
      ti_Data.append(tll_Data[j+1])

  for i in range(Num_Pkts):
      d = ts_Data[i] - tll_Data[i]                  # Tiempo de los clientes en el sistema
      r_Data.append(d)

      c = ti_Data[i] - tll_Data[i]                  # Tiempo en la cola
      if c>0:
        tc_Data.append(c)
      else:
        tc_Data.append(0)

  W = mean(r_Data)                                  # Tiempo medio de estancia de los clientes en el sistema
  Wq = mean(tc_Data)                                # Tiempo medio de espera de los clientes en la cola

  L = lamda * W                                     # Número medio de clientes en el sistema
  Lq = lamda * Wq                                   # Número medio de clientes en la cola

  return (W,Wq,L,Lq)

### b) ¿Cuál es la longitud media de la cola?

In [27]:
W,Wq,L,Lq=mm1_var(2/15,1/5,100)

print('El número medio de clientes en la cola es Lq=%.2f' %round(Lq, 2))

El número medio de clientes en la cola es Lq=1.34


### c) ¿Cuál es el tiempo medio que un cliente pasa en la biblioteca hasta que le han atendido?

In [28]:
print('El tiempo medio de estancia de los clientes en la biblioteca hasta ser atendidos es W=%.2f' %round(W, 2))

El tiempo medio de estancia de los clientes en la biblioteca hasta ser atendidos es W=14.82


### d) ¿Cuál es el tiempo medio que un cliente pasa en la cola esperando a que le atiendan?

In [29]:
print('El tiempo medio de espera de los clientes en la cola es Wq=%.2f' %round(Wq, 2))

El tiempo medio de espera de los clientes en la cola es Wq=10.07


### e) El encargado de la biblioteca pasa su tiempo en que no hay clientes clasificando artículos de revistas. Si puede clasificar 22 fichas por hora como media cuando trabaja continuamente, ¿cuántas fichas puede ordenar durante su trabajo?

Realizaremos una funcion que nos devuelve la lista de:


*   Tiempo de llegada
*   Tiempo en la cola
*   Tiempo de inicio del servicio
*   Tiempo de salida





In [30]:
def mm1(lamda,mu,Num_Pkts):
  count = 0                           # Contador de paquetes simulados
  t = 0
  N = 0                               # Variable de estado: numero de paquetes en el sistema

  tll = random.expovariate(lamda)     # Tiempo de llegada
  ts = Infinity                       # Tiempo de salida

  t_evento_anterior = 0.0             # tiempo del último evento


  # Variables de salida
  tll_Data = []                       # Lista de tiempos de llegada
  ts_Data = []                        # Lista de tiempos de salida
  ti_Data = []                        # Lista de tiempos de inicio de servicio
  tc_Data = []                        # Lista de tiempos en la cola
  r_Data = []                         # Lista de retrasos individuales

  ti_Data.append(tll)

  while count < Num_Pkts:
      if tll < ts:                                  # Evento de llegada
          t = tll
          tll_Data.append(t)
          
          t_evento_anterior = t
          
          N = N + 1.0
          tll = t + random.expovariate(lamda) 
          if N == 1:
              ts = t + random.expovariate(mu)

      else:                                         # Evento de salida
          t = ts
          ts_Data.append(t)

          t_evento_anterior = t

          N = N - 1.0
          count = count + 1                         # Paquetes simulados
          if N > 0:
              ts = t + random.expovariate(mu)
          else:
              ts = Infinity

  for j in range(Num_Pkts-1):                         # Tiempo de inicio del servicio
    if tll_Data[j+1]<ts_Data[j]:
      ti_Data.append(ts_Data[j])
    else:
      ti_Data.append(tll_Data[j+1])
              
  for i in range(Num_Pkts):
      d = ts_Data[i] - tll_Data[i]                  # Tiempo de los clientes en el sistema
      r_Data.append(d)

      c = ti_Data[i] - tll_Data[i]                  # Tiempo en la cola
      if c>0:
        tc_Data.append(c)
      else:
        tc_Data.append(0)

  return (tll_Data,tc_Data,ti_Data,ts_Data)

Primero obtendremos los tiempos en los que no se tienen clientes, para sumarlos todos y obtener la cantidad de fichas que realizara en ese tiempo.

Sabemos que la media de fichas que realiza es de $\frac{22 \text{ fichas}}{1 \text{ hora}}$, pero debemos pasarlo a minutos ya que nuestro programa ocupa esa medida, entonces:
 
$\frac{22 \text{ fichas}}{1 \text{ hora}}=\frac{22 \text{ fichas}}{1 \text{ hora}}\frac{1 \text{ hora}}{60 \text{ minutos}}=\frac{11 \text{ fichas}}{30 \text{ minuto}}$

In [31]:
tll,tc,ti,ts=mm1(2/15,1/5,100)

#print(tll,'\n',tc,'\n',ti,'\n',ts)
#print(len(tll),'\n',len(tc),'\n',len(ti),'\n',len(ts))

tsc=[]                              # Lista de tiempos sin clientes
tsc.append(tll[0])                  # Se le asigna el primer valor de llegada, ya que antes de esta llegada no hay clientes

for i in range(len(ts)-1):
  temp=tll[i+1]-ts[i]

  if temp>0:
    tsc.append(temp)
  else:
    tsc.append(0)
    
#print(tsc)
#print(len(tsc))
stsc=sum(tsc)                      # Suma de todos los tiempos sin clientes

nf=stsc*(11/30)

print('%.2f es el número de fichas que se hicieron' %nf, 'en los %.2f minutos sin clientes' %stsc)

116.26 es el número de fichas que se hicieron en los 317.07 minutos sin clientes


## 2. Lavado de autos

Un pequeño autoservicio de lavado que solo tienen capacidad para atender de uno en uno a los autos que llegan, tiene un aparcamiento para 10 coches, incluyendo el que está siendo lavado. La empresa ha estimado que los coches llegan siguiendo una distribución de Poisson con una media de 20 coches/hora, el tiempo de servicio sigue una distribución exponencial de 12 minutos. La empresa abre durante 10 horas al día.

¿Cuál es la media de coches perdidos cada día debido a las limitaciones de espacio?

La siguiente funcion nos regresa:

*   Número de clientes en el sistema en cada momento del dia
*   Número de clientes perdidos en cada momento del dia
*   Número total de clientes perdidos al dia

In [109]:
def mm1_n(lamda,mu,Num_Pkts,capacidadsistema):
  count = 0                           # Contador de paquetes simulados
  t = 0
  N = 0                               # Variable de estado: numero de paquetes en el sistema
  ch = 0
  chp = 0

  tll = random.expovariate(lamda)     # Tiempo de llegada
  ts = Infinity                       # Tiempo de salida

  t_evento_anterior = 0.0             # tiempo del último evento


  # Variables de salida
  tll_Data = []                       # Lista de tiempos de llegada
  ts_Data = []                        # Lista de tiempos de salida
  ti_Data = []                        # Lista de tiempos de inicio de servicio
  tc_Data = []                        # Lista de tiempos en la cola
  r_Data = []                         # Lista de retrasos individuales
  n_Data = []                         # Lista de numero de clientes en el sistema
  np_Data = []                        # Lista de numero de clientes perdidos

  ti_Data.append(tll)
  n_Data.append(ch)
  np_Data.append(chp)

  while count < Num_Pkts:
      if tll < ts:                                  # Evento de llegada
          t = tll
          tll_Data.append(t)
          
          t_evento_anterior = t
          
          N = N + 1.0
          tll = t + random.expovariate(lamda) 
          if N == 1:
              ts = t + random.expovariate(mu)

      else:                                         # Evento de salida
          t = ts
          ts_Data.append(t)

          t_evento_anterior = t

          N = N - 1.0
          count = count + 1                         # Paquetes simulados
          if N > 0:
              ts = t + random.expovariate(mu)
          else:
              ts = Infinity

  for j in range(Num_Pkts-1):                         # Tiempo de inicio del servicio
    if tll_Data[j+1]<ts_Data[j]:
      ti_Data.append(ts_Data[j])
    else:
      ti_Data.append(tll_Data[j+1])
          
  for i in range(Num_Pkts):
      d = ts_Data[i] - tll_Data[i]                  # Tiempo de los clientes en el sistema
      r_Data.append(d)

      c = ti_Data[i] - tll_Data[i]                  # Tiempo en la cola
      if c>0:
        tc_Data.append(c)
      else:
        tc_Data.append(0)

  j=0
  k=0
  while (j<Num_Pkts):                               # Numero de clientes en el sistema
    if tll_Data[j]<ts_Data[k]:                  # Tiempo de llegada es menor al tiempo de salida

      if ch<capacidadsistema:
        ch=ch+1                                     # Llegada de un cliente
        np_Data.append(0)
      else:
        chp=chp+1                                   # Clientes perdidos
        np_Data.append(1)

      n_Data.append(ch)
      j=j+1                                     # Cambia al siguiente tiempo de llegada
    else:
      ch=ch-1                                       # Salida de un cliente
      n_Data.append(ch)
      np_Data.append(0)
      k=k+1                                     # Cambia al siguiente tiempo de salida

  return (n_Data,chp)

Como las llegadas tienen una distribucion poisson, los servicios una distribucion exponecial y solo tenemos un servidor, tenemos un modelo de la forma: **M/M/1**

El promedio de llegadas es de $λ=\frac{20\text{ llegadas}}{1\text{ hora}}$, pero haremos la conversion a llegadas por minuto.

$λ=\frac{20\text{ llegadas}}{1\text{ hora}}(\frac{1\text{ hora}}{60 \text{ minuto}})=\frac{1}{3} \frac{\text{llegadas}}{\text{minuto}}$

El promedio de servicios es de $μ=\frac{1 \text{ llegada}}{12 \text{ minuto}}$

Sabemos que el tiempo de servicio es de $10$ horas, lo que en minutos es $600$ minutos, entonces el número total de clientes que podemos recibir durante todo el dia es de:

$(600 \text{ minuto})(\frac{1}{3} \frac{\text{llegadas}}{\text{minuto}})=100 \text{ llegadas}$

Al solo tener un servidor sabemos que si llegan más de un usuario a la vez tendremos cola y el tamaño de cola que podemos tener sin perder clientes es de $9$.

Ahora realizaremos el calculo de la media de coches perdidos repitiendo 30 veces el calculo usando la funcion anterir.

In [111]:
mcp = []                        # Lista de numero de clientes perdidos

for i in range(30):
  n,cp=mm1_n(1/3,1/12,100,10)
  mcp.append(cp)

print('La media de coches perdidos cada día debido a las limitaciones de espacio es %.2f' %mean(mcp))

La media de coches perdidos cada día debido a las limitaciones de espacio es 65.97


## 3. John Macko

John Macko es alumno en la U de Ozark. Hace trabajos extraños para aumentar sus ingresos. Las peticiones de trabajo llegan en promedio cada 5 días, pero el tiempo entre ellas es exponencial. El tiempo para terminar un trabajo también es exponencial, con una media de 4 días.






Como las llegadas tienen una distribucion exponencial, el tiempo para terminar una distribucion exponecial y solo tenemos un servidor al solo realizar el trabajo John, tenemos un modelo de la forma: **M/M/1**

El promedio de llegadas es de $λ=\frac{1\text{ trabajo}}{5\text{ dia}}$.

El promedio de terminar el trabajo es de $μ=\frac{1 \text{ trabajo}}{12 \text{ dia}}$

No tenemos mas especificaciones del problema supondremos que llegan 30 trabajos. 

### a) ¿Cuál es la probabilidad de que le falte trabajo a John?

La probabilidad de que le falte trabajo es cuando obtenemos la probabilidad de 0 trabajos

$P(trabajos=0)$

In [107]:
pt=pmm1_ll(1/5,1/12,30)

print("Probabilidad de que le falte trabajo es de %.4f" %pt[1])

Probabilidad de que le falte trabajo es de 0.0242


### b) Si John cobra unos $50 por cada trabajo, ¿cuál es su ingreso mensual promedio?

In [124]:
tm = []                        # Lista de numero de trabajos realizados

for i in range(30):            # Se realiza 30 veces el calculo de cuantos trabajos se tienen al di, porque se necesitar el calculo mensual.
  n,tp=mm1_n(1/5,1/12,30,1)
  st=sum(n)                    # Trabajos al dia
  tm.append(st)

print('Numero medio de trabajos al mes es de %.2f' %mean(tm),'y su ingreso mensual promedio seria de $%.2f' %(50*mean(tm)))

Numero medio de trabajos al mes es de 12.57 y su ingreso mensual promedio seria de $628.33


### c) Si al final del semestre John decide subcontratar los trabajos pendientes a $40 cada uno, ¿cuánto debe esperar pagar en promedio?

In [128]:
tpm = []                       # Lista de numero de trabajos pendientes

for i in range(180):           # Se realiza 180 veces el calculo de cuantos trabajos pendientes se tienen al dia, porque se necesitar el calculo del semestre.
  n,tp=mm1_n(1/5,1/12,30,1)
  tpm.append(tp)

print('Numero medio de trabajos pendientes a final del semestre es de %.2f' %mean(tpm),'y espera pagar en promedio seria de $%.2f' %(40*mean(tpm)))

Numero medio de trabajos pendientes a final del semestre es de 17.33 y espera pagar en promedio seria de $693.11
